# 04_agent_prototype.ipynb
## (Refactorizado) Prototipo del Pipeline Completo (Pasos 1-6)

**Objetivo:** Orquestar el pipeline completo de RAG Multimodal en un solo lugar, desde la ingesta de archivos hasta la consulta del agente.

**Flujo:**
1.  **FASE 1: INDEXACIÓN**
    -   Paso 1: Convertir Excel -> PNG
    -   Paso 2: Extraer Markdown (PDFs y PNGs) con API ADE
    -   Paso 3: Convertir Markdown -> JSON (con LLM)
    -   Paso 4: Generar Embeddings (con CLIP)
    -   Paso 5: Indexar en ChromaDB
2.  **FASE 2: CONSULTA**
    -   Paso 6: Probar el Agente RAG

In [2]:
import sys
import logging
from pathlib import Path
import json
import time

# --- Celda de Configuración Estándar ---
PROJECT_ROOT = Path.cwd().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

# 1. Importar la config (esto carga el .env automáticamente)
import src.utils.config 
from src.utils.config import OPENAI_API_KEY, LANDING_AI_API_KEY

# 2. Configurar logging básico para el notebook
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print(f"Raíz del proyecto establecida en: {PROJECT_ROOT}")
print("✅ Entorno configurado. Logging listo.")

# 3. Verificar API Keys (Mejor Práctica del Análisis)
if not LANDING_AI_API_KEY or "..." in LANDING_AI_API_KEY:
    logger.error("Falta LANDING_AI_API_KEY en .env. El Paso 2 fallará.")
if not OPENAI_API_KEY or "..." in OPENAI_API_KEY:
    logger.error("Falta OPENAI_API_KEY en .env. El Paso 3 y 6 fallarán.")

Raíz del proyecto establecida en: c:\Users\Usuario\Documents\UTEC\Liquidaciones Agent\multidoc-agent
✅ Entorno configurado. Logging listo.


## FASE 1: INDEXACIÓN (Pasos 1-5)

### Paso 1: Convertir Excel -> PNG

In [2]:
from src.preprocessors import process_all_excels

print("Iniciando [Paso 1]... (Excel -> PNG)")
start_time = time.time()
excel_results = process_all_excels()
elapsed = time.time() - start_time
print(f"  Resultado: {len([r for r in excel_results if r.get('status') == 'success'])} Excel procesados en {elapsed:.2f}s")

INFO:src.preprocessors.excel_to_image:Encontrados 2 archivos Excel
INFO:src.preprocessors.excel_to_image:Iniciando conversión de Excel (en chunks): c:\Users\Usuario\Documents\UTEC\Liquidaciones Agent\multidoc-agent\data\input\excel\10841- INFORME GENERAL - MN - SKY KNIGHT  -  LAS BAMBAS   - 09 -10 -2025 (1).xlsx


Iniciando [Paso 1]... (Excel -> PNG)


INFO:src.preprocessors.excel_to_image:Usando rango total: B1:JC223
INFO:src.preprocessors.excel_to_image:Excel procesado y guardado como 8 chunks PNG.
INFO:src.preprocessors.excel_to_image:Metadata de Excel guardada: 10841- INFORME GENERAL - MN - SKY KNIGHT  -  LAS BAMBAS   - 09 -10 -2025 (1)_metadata.json
INFO:src.preprocessors.excel_to_image:Iniciando conversión de Excel (en chunks): c:\Users\Usuario\Documents\UTEC\Liquidaciones Agent\multidoc-agent\data\input\excel\~$10841- INFORME GENERAL - MN - SKY KNIGHT  -  LAS BAMBAS   - 09 -10 -2025 (1).xlsx
ERROR:src.preprocessors.excel_to_image:Error al convertir Excel a imagen: [Errno 13] Permission denied: 'c:\\Users\\Usuario\\Documents\\UTEC\\Liquidaciones Agent\\multidoc-agent\\data\\input\\excel\\~$10841- INFORME GENERAL - MN - SKY KNIGHT  -  LAS BAMBAS   - 09 -10 -2025 (1).xlsx'
Traceback (most recent call last):
  File "c:\Users\Usuario\Documents\UTEC\Liquidaciones Agent\multidoc-agent\src\preprocessors\excel_to_image.py", line 35, in

  Resultado: 1 Excel procesados en 50.61s


### Paso 2: Extraer Markdown (API ADE)

In [3]:
from src.extractors import process_all_documents

print("Iniciando [Paso 2]... (PDF/PNG -> Markdown con API ADE)")
start_time = time.time()
ade_results = process_all_documents()
elapsed = time.time() - start_time
print(f"  Resultado: {len([r for r in ade_results if r.get('status') == 'success'])} documentos procesados por ADE en {elapsed:.2f}s")

c:\Users\Usuario\Documents\UTEC\Liquidaciones Agent\multidoc-agent\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Iniciando [Paso 2]... (PDF/PNG -> Markdown con API ADE)
2025-11-08 11:44:21,222 - src.extractors.ocr_extractor - INFO - Iniciando PASO 3: Extracción de Markdown (ADE) de todos los documentos...


INFO:src.extractors.ocr_extractor:Iniciando PASO 3: Extracción de Markdown (ADE) de todos los documentos...


2025-11-08 11:44:21,224 - src.extractors.ocr_extractor - INFO - Extractor OCR iniciado con el proveedor: landing_ai


INFO:src.extractors.ocr_extractor:Extractor OCR iniciado con el proveedor: landing_ai


...

2025-11-08 11:48:17,063 - src.extractors.ocr_extractor - INFO - Markdown guardado en: 10841- INFORME GENERAL - MN - SKY KNIGHT  -  LAS BAMBAS   - 09 -10 -2025 (1)_chunk_r50_c250_text.json


INFO:src.extractors.ocr_extractor:Markdown guardado en: 10841- INFORME GENERAL - MN - SKY KNIGHT  -  LAS BAMBAS   - 09 -10 -2025 (1)_chunk_r50_c250_text.json


2025-11-08 11:48:17,067 - src.extractors.ocr_extractor - INFO - Extracción de Markdown completada. 9 archivos procesados.


INFO:src.extractors.ocr_extractor:Extracción de Markdown completada. 9 archivos procesados.


  Resultado: 9 documentos procesados por ADE en 235.85s


### Paso 3: Convertir Markdown -> JSON (LLM)

In [4]:
from src.extractors import process_all_extracted_text

print("Iniciando [Paso 3]... (Markdown -> JSON con LLM)")
start_time = time.time()
parser_results = process_all_extracted_text()
elapsed = time.time() - start_time
print(f"  Resultado: {len([r for r in parser_results if r.get('status') == 'success'])} JSONs estructurados en {elapsed:.2f}s")

Iniciando [Paso 3]... (Markdown -> JSON con LLM)
2025-11-08 11:48:35,821 - src.extractors.structure_parser - INFO - Iniciando PASO 4: Parseo y Estructuración (Markdown a JSON)...


INFO:src.extractors.structure_parser:Iniciando PASO 4: Parseo y Estructuración (Markdown a JSON)...


2025-11-08 11:48:37,619 - src.extractors.structure_parser - INFO - StructureParser inicializado con el esquema: LiquidacionData


INFO:src.extractors.structure_parser:StructureParser inicializado con el esquema: LiquidacionData


2025-11-08 11:48:37,622 - src.extractors.structure_parser - INFO - Se encontraron 9 archivos de Markdown para parsear.


...

2025-11-08 11:50:31,894 - src.extractors.structure_parser - INFO - Estructura JSON guardada en: 25105 25106 25107 & 25108 MV SKY KNIGHT - VAL TISUR_structure.json


INFO:src.extractors.structure_parser:Estructura JSON guardada en: 25105 25106 25107 & 25108 MV SKY KNIGHT - VAL TISUR_structure.json


2025-11-08 11:50:31,897 - src.extractors.structure_parser - INFO - Parseo completado: 9 archivos procesados.


INFO:src.extractors.structure_parser:Parseo completado: 9 archivos procesados.


  Resultado: 8 JSONs estructurados en 116.08s


### Paso 4: Generar Embeddings (CLIP)

In [5]:
from src.embeddings import process_all_multimodal

print("Iniciando [Paso 4]... (Generando Embeddings con CLIP)")
start_time = time.time()
embedding_results = process_all_multimodal()
elapsed = time.time() - start_time
print(f"  Resultado: {embedding_results.get('total_embeddings', 0)} embeddings generados en {elapsed:.2f}s")
print(f"     Imágenes: {embedding_results.get('image_count', 0)}")
print(f"     Textos (JSON): {embedding_results.get('text_count', 0)}")

Iniciando [Paso 4]... (Generando Embeddings con CLIP)
2025-11-08 11:50:45,685 - src.embeddings.clip_encoder - INFO - Cargando modelo CLIP: openai/clip-vit-base-patch32


INFO:src.embeddings.clip_encoder:Cargando modelo CLIP: openai/clip-vit-base-patch32


2025-11-08 11:50:45,687 - src.embeddings.clip_encoder - INFO - Usando device: cpu


INFO:src.embeddings.clip_encoder:Usando device: cpu
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


2025-11-08 11:50:50,752 - src.embeddings.clip_encoder - INFO - CLIP encoder cargado exitosamente


...

2025-11-08 11:50:57,217 - src.embeddings.clip_encoder - INFO - Embeddings guardados: c:\Users\Usuario\Documents\UTEC\Liquidaciones Agent\multidoc-agent\data\embeddings\image_embeddings.json


INFO:src.embeddings.clip_encoder:Embeddings guardados: c:\Users\Usuario\Documents\UTEC\Liquidaciones Agent\multidoc-agent\data\embeddings\image_embeddings.json


2025-11-08 11:50:57,235 - src.embeddings.clip_encoder - INFO - Embeddings guardados: c:\Users\Usuario\Documents\UTEC\Liquidaciones Agent\multidoc-agent\data\embeddings\text_embeddings.json


INFO:src.embeddings.clip_encoder:Embeddings guardados: c:\Users\Usuario\Documents\UTEC\Liquidaciones Agent\multidoc-agent\data\embeddings\text_embeddings.json


  Resultado: 16 embeddings generados en 11.62s
     Imágenes: 8
     Textos (JSON): 8


### Paso 5: Indexar en ChromaDB

In [3]:
from src.vectorstore import index_all_embeddings, MultimodalIndexer

print("Iniciando [Paso 5]... (Indexando en ChromaDB)")
start_time = time.time()
index_results = index_all_embeddings()
elapsed = time.time() - start_time
print(f"  Resultado: {index_results.get('total_indexed', 0)} documentos indexados en {elapsed:.2f}s")

# Verificar estadísticas (como en el análisis)
try:
    indexer = MultimodalIndexer()
    stats = indexer.get_collection_stats()
    print("\n--- Estadísticas de ChromaDB ---")
    print(json.dumps(stats, indent=2))
    print("---------------------------------")
except Exception as e:
    logger.error(f"Error al obtener estadísticas de ChromaDB: {e}")

c:\Users\Usuario\Documents\UTEC\Liquidaciones Agent\multidoc-agent\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:src.vectorstore.multimodal_indexer:Iniciando PASO 5: Indexación de todos los embeddings...
INFO:src.vectorstore.multimodal_indexer:No se proveyó ClipEncoder, creando instancia por defecto...


Iniciando [Paso 5]... (Indexando en ChromaDB)
2025-11-08 12:13:26,186 - src.embeddings.clip_encoder - INFO - Cargando modelo CLIP: openai/clip-vit-base-patch32


INFO:src.embeddings.clip_encoder:Cargando modelo CLIP: openai/clip-vit-base-patch32


2025-11-08 12:13:26,188 - src.embeddings.clip_encoder - INFO - Usando device: cpu


INFO:src.embeddings.clip_encoder:Usando device: cpu
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


...

2025-11-08 12:13:33,140 - src.embeddings.clip_encoder - INFO - Usando device: cpu


INFO:src.embeddings.clip_encoder:Usando device: cpu


2025-11-08 12:13:36,472 - src.embeddings.clip_encoder - INFO - CLIP encoder cargado exitosamente


INFO:src.embeddings.clip_encoder:CLIP encoder cargado exitosamente
INFO:src.vectorstore.multimodal_indexer:Instancia de ClipEncoder por defecto creada.
INFO:src.vectorstore.multimodal_indexer:No se proveyó ChromaManager, usando get_chroma_manager() por defecto...
INFO:src.vectorstore.multimodal_indexer:Instancia de ChromaManager por defecto obtenida.
INFO:src.vectorstore.multimodal_indexer:MultimodalIndexer inicializado y listo.
INFO:src.vectorstore.multimodal_indexer:Obteniendo estadísticas de la colección 'multidoc_collection'...
INFO:src.vectorstore.multimodal_indexer:Estadísticas obtenidas: {'total_documents': 8, 'image_documents': 8, 'text_documents': 0, 'multimodal': False}



--- Estadísticas de ChromaDB ---
{
  "total_documents": 8,
  "image_documents": 8,
  "text_documents": 0,
  "multimodal": false
}
---------------------------------


## FASE 2: CONSULTA (Paso 6)

### Paso 6: Probar el Agente RAG

In [5]:
from src.agent import run_agent_query

print("Iniciando [Paso 6]... (Prueba de consulta del Agente)")
query = "¿Cuál es el tonelaje para el lote 25105?"
print(f"\nQuery: '{query}'")

start_time = time.time()
result = run_agent_query(query)
elapsed = time.time() - start_time

print(f"\n--- Respuesta del Agente ({elapsed:.2f}s) ---")
# Imprimir la respuesta formateada
if result.get('status') == 'success':
    response_data = result.get('response', {})
    print(f"Respuesta: {response_data.get('answer', 'Sin respuesta.')}")
    print(f"Confianza: {response_data.get('quality_score', 0) * 100:.1f}%")
    print(f"Fuentes usadas ({len(response_data.get('sources', []))}):\n{json.dumps(response_data.get('sources', []), indent=2)}")
else:
    print(f"❌ Error del Agente: {result.get('error', 'Error desconocido')}")
print("---------------------------------")

Iniciando [Paso 6]... (Prueba de consulta del Agente)

Query: '¿Cuál es el tonelaje para el lote 25105?'
2025-11-08 12:14:08,958 - src.agent.graph_agent - INFO - Ejecutando agente con query: ¿Cuál es el tonelaje para el lote 25105?


INFO:src.agent.graph_agent:Ejecutando agente con query: ¿Cuál es el tonelaje para el lote 25105?


2025-11-08 12:14:08,967 - src.agent.nodes - INFO - Query node - Query: ¿Cuál es el tonelaje para el lote 25105?


INFO:src.agent.nodes:Query node - Query: ¿Cuál es el tonelaje para el lote 25105?


2025-11-08 12:14:08,972 - src.agent.nodes - INFO - Query preparada: ¿Cuál es el tonelaje para el lote 25105?


...

2025-11-08 12:14:15,927 - src.agent.nodes - INFO - Respuesta formateada


INFO:src.agent.nodes:Respuesta formateada


2025-11-08 12:14:15,930 - src.agent.graph_agent - INFO - Ejecución del agente completada


INFO:src.agent.graph_agent:Ejecución del agente completada



--- Respuesta del Agente (6.98s) ---
Respuesta: No se encontraron documentos relevantes
Confianza: 70.0%
Fuentes usadas (0):
[]
---------------------------------


### Fin del Prototipo
Si todas las celdas se ejecutaron, el pipeline completo funciona.

**Próximo paso:** `05_agent_query.ipynb` para consultas interactivas.